In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime

# Plan du notebook:
1 Lecture des données et première exploration  
 * 1.1 Dataframe customers  
 * 1.2 Dataframe products  
 * 1.3 Dataframe transactions  
 
2 Nettoyage des Dataframes  
 * 2.1 nettoyage de df_transac  
 * 2.2 nettoyage de df_products  
 
3 Rapprochement des différents export  
* 3.1 jointure entre df_transac et df_products  
  * 3.1.1 jointure et vérification des données  
  * 3.1.2 traitement des erreurs suite à la jointure  
* 3.2 jointure entre df_trans_prod et df_custom  
  * 3.2.1 jointure et vérification des données  
  * 3.2.2 traitement des erreurs suite jointure  

4 ajouts de colonne au dataframe  
 * 4.1 ajout d'un colonne age des clients  
 * 4.2 ajout d'une colonne classe d'age  
 
5 création d'un dataframe pour pouvoir répondre aux différentes demandes

# 1 Lecture des données et première exploration

## 1.1 dataframe customers

In [2]:
# import des données consommateurs
df_custom = pd.read_csv("customers.csv")
df_custom

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


In [3]:
# observation de la taille du dataframe
df_custom.shape

(8623, 3)

In [4]:
# observation des infos contenues dans le dataframe
df_custom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [5]:
# observation de la description du dataframe
df_custom.describe(include= "all")

,client_id,sex,birth
count,8623,8623,8623.000000
unique,8623,2,NaN
top,c_4410,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,1978.280877
std,NaN,NaN,16.919535
min,NaN,NaN,1929.000000
25%,NaN,NaN,1966.000000
50%,NaN,NaN,1979.000000
75%,NaN,NaN,1992.000000


In [6]:
# vérification de la présence de valeurs nuls
df_custom.isnull().sum()

client_id    0
sex          0
birth        0
dtype: int64

In [7]:
# vérification que client_id ne contient que des valeurs uniques et peut servir de clé primaire
df_custom['client_id'].nunique() == len(df_custom)

True

le df contient 8623 lignes et 3 colonnes  
le client_id est bien une valeur unique et peut servir de clefs primaires  
il n'y a pas de valeur manquante

## 1.2 dataframe products

In [8]:
# import des données produits
df_prod = pd.read_csv("products.csv")
df_prod

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
...,...,...,...
3282,2_23,115.99,2
3283,0_146,17.14,0
3284,0_802,11.22,0
3285,1_140,38.56,1


In [9]:
# observation de la taille du dataframe
df_prod.shape

(3287, 3)

In [10]:
# observation des infos contenues dans le dataframe
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [11]:
# observation de la description du dataframe
df_prod.describe()

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


In [12]:
# vérification de la présence de valeurs nuls
df_prod.isnull().sum()

id_prod    0
price      0
categ      0
dtype: int64

In [13]:
# vérification que client_id ne contient que des valeurs uniques et peut servir de clé primaire
df_prod['id_prod'].nunique() == len(df_prod)

True

In [14]:
# recherche des prix negatifs dans le dataframe
df_prod[df_prod["price"]<0]

,id_prod,price,categ
731,T_0,-1.0,0


Il y a 3287 lignes et 3 colonnes  
Il n'y a pas de valeur manquante  
Le id_prod est bien une valeur unique et peut servir de clef primaire  
Il y a un prix négatif: erreur à vérifier

## 1.3 dataframe transactions

In [15]:
# import des données transactions
df_transac = pd.read_csv("transactions.csv")
df_transac.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [16]:
# observation de la taille du dataframe
df_transac.shape

(679532, 4)

In [17]:
# observation des infos contenues dans le dataframe
df_transac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679532 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679532 non-null  object
 1   date        679532 non-null  object
 2   session_id  679532 non-null  object
 3   client_id   679532 non-null  object
dtypes: object(4)
memory usage: 20.7+ MB


In [18]:
# observation de la description du dataframe
df_transac.describe(include = "all")

,id_prod,date,session_id,client_id
count,679532,679532,679532,679532
unique,3267,679371,342316,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,2252,13,200,25488


Il y a eut une phase de test sous le nom session_id s_0 reconnaissable au fait que les dates indiquent test  

# 2 Nettoyage des Dataframes

## 2.1 nettoyage de df_transac

In [19]:
# observation de la session test
df_test = df_transac.loc[df_transac["session_id"] == "s_0"]
df_test

,id_prod,date,session_id,client_id
3019,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
5138,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0
9668,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
10728,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
15292,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0
...,...,...,...,...
657830,T_0,test_2021-03-01 02:30:02.237417,s_0,ct_0
662081,T_0,test_2021-03-01 02:30:02.237427,s_0,ct_1
670680,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1
671647,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_1


In [20]:
# identification des référence client utilisé pour le test
df_test["client_id"].nunique()

2

In [21]:
# identification des référeces produit utilisé pour le test
df_test["id_prod"].nunique()

1

In [22]:
# vérification de la présence de doublon
df_transac.loc[df_transac["date"].duplicated(keep = False)]

,id_prod,date,session_id,client_id
3019,T_0,test_2021-03-01 02:30:02.237419,s_0,ct_0
5138,T_0,test_2021-03-01 02:30:02.237425,s_0,ct_0
9668,T_0,test_2021-03-01 02:30:02.237437,s_0,ct_1
10728,T_0,test_2021-03-01 02:30:02.237436,s_0,ct_0
15292,T_0,test_2021-03-01 02:30:02.237430,s_0,ct_0
...,...,...,...,...
657830,T_0,test_2021-03-01 02:30:02.237417,s_0,ct_0
662081,T_0,test_2021-03-01 02:30:02.237427,s_0,ct_1
670680,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1
671647,T_0,test_2021-03-01 02:30:02.237424,s_0,ct_1


Le produit qui à un prix négatif à été utilisé pour faire le test.  
Il faut retirer l'id client et produit utilisé pour le test dans df_prod et df_custom

In [23]:
# retrait des données du test
df_transac = df_transac[df_transac["id_prod"] != "T_0"]
df_transac.describe(include="all")

,id_prod,date,session_id,client_id
count,679332,679332,679332,679332
unique,3266,679332,342315,8600
top,1_369,2022-05-20 13:21:29.043970,s_118668,c_1609
freq,2252,1,14,25488


In [24]:
# changement du format de la colonne date du format objet au format date
df_transac["date"] = pd.to_datetime(df_transac["date"], errors = "coerce")
df_transac

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232
...,...,...,...,...
679527,0_1551,2022-01-15 13:05:06.246925,s_150195,c_8489
679528,1_639,2022-03-19 16:03:23.429229,s_181434,c_4370
679529,0_1425,2022-12-20 04:33:37.584749,s_314704,c_304
679530,0_1994,2021-07-16 20:36:35.350579,s_63204,c_2227


In [25]:
# vérification de la suppression des ligne test
df_transac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 25.9+ MB


# 2.2 nettoyage du df_products

In [26]:
# retrait du produit dont le prix est négatif car il a servit au test
df_prod = df_prod[df_prod['price']!= -1]

In [27]:
# vérification de la suppression de la ligne au prix négatif
df_prod.describe()

,price,categ
count,3286.000000,3286.000000
mean,21.863597,0.370359
std,29.849786,0.615446
min,0.620000,0.000000
25%,6.990000,0.000000
50%,13.075000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


## 2.3 nettoyage du df_custom

In [28]:
# retrait des id client test
df_custom = df_custom[~df_custom["client_id"].isin(["ct_0","ct_1"])]
df_custom

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


In [29]:
# vérification de la suppression des clients test
df_custom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8621 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8621 non-null   object
 1   sex        8621 non-null   object
 2   birth      8621 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 269.4+ KB


# 3 Rapprochement des différents exports

## 3.1 jointure entre df_transac et df_products

### 3.1.1 jointure et vérification des données

In [30]:
# jointure externe sur la colonne commune id_prod
df_trans_prod = pd.merge(df_transac, df_prod, on="id_prod", how="outer")
df_trans_prod

,id_prod,date,session_id,client_id,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0.0
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0.0
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0.0
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0.0
...,...,...,...,...,...,...
679348,0_1624,NaT,NaN,NaN,24.50,0.0
679349,2_86,NaT,NaN,NaN,132.36,2.0
679350,0_299,NaT,NaN,NaN,22.99,0.0
679351,0_510,NaT,NaN,NaN,23.66,0.0


In [31]:
# observation des valeurs nul
df_trans_prod.isnull().sum()

id_prod         0
date           21
session_id     21
client_id      21
price         221
categ         221
dtype: int64

In [32]:
# observation du dataframe des prix nuls
df_nul_trpr = df_trans_prod[df_trans_prod["price"].isnull()]
df_nul_trpr

,id_prod,date,session_id,client_id,price,categ
542560,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,NaN,NaN
542561,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,NaN,NaN
542562,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,NaN,NaN
542563,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,NaN,NaN
542564,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,NaN,NaN
...,...,...,...,...,...,...
542776,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,NaN,NaN
542777,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,NaN,NaN
542778,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,NaN,NaN
542779,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,NaN,NaN


In [33]:
# vérif des valeurs existant dans id_prod du df_nul_trpr
df_nul_trpr["id_prod"].unique()

array(['0_2245'], dtype=object)

In [34]:
# vérif des valeurs existant dans categ du df_nul_trpr
df_nul_trpr["categ"].unique()

array([nan])

Il y a des commandes où les prix et catégorie sont manquant  
Pour compléter categ il suffit d'utiliser le début de l'id_prod car celui-ci est composé d'abord de la categ et d'un chiffre après l'underscore  
Pour le prix il faudra aussi compléter (methode utilisé médian)

In [35]:
# observation des valeurs nul de cession
df_nul_session = df_trans_prod[df_trans_prod["session_id"].isnull()]
df_nul_session

,id_prod,date,session_id,client_id,price,categ
679332,0_1016,NaT,NaN,NaN,35.06,0.0
679333,0_1780,NaT,NaN,NaN,1.67,0.0
679334,0_1062,NaT,NaN,NaN,20.08,0.0
679335,0_1119,NaT,NaN,NaN,2.99,0.0
679336,0_1014,NaT,NaN,NaN,1.15,0.0
679337,1_0,NaT,NaN,NaN,31.82,1.0
679338,0_1318,NaT,NaN,NaN,20.92,0.0
679339,0_1800,NaT,NaN,NaN,22.05,0.0
679340,0_1645,NaT,NaN,NaN,2.99,0.0
679341,0_322,NaT,NaN,NaN,2.99,0.0


les produits n'ont pas fait l'objet de vente sur le site c'est pour cela qu'il n'y a pas de session/client d'indiqué

### 3.1.2 traitement des erreurs suite à la jointure

#### traitement de la colonne categ

In [36]:
# remplacement valeur manquante dans categ par zéro
df_trans_prod["categ"] = df_trans_prod["categ"].fillna(0)

In [37]:
# verif que les valeurs nulles de categ ont été remplacé
df_trans_prod.isnull().sum()

id_prod         0
date           21
session_id     21
client_id      21
price         221
categ           0
dtype: int64

#### traitement de la colonne prix

In [38]:
# les produits sont divisé par catégorie
# les prix manquants sont de la categ 0
df_categ_zero = df_prod[df_prod["categ"] == 0]
df_categ_zero

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
4,0_1507,3.99,0
5,0_1163,9.99,0
...,...,...,...
3280,0_1314,20.63,0
3281,0_607,14.99,0
3283,0_146,17.14,0
3284,0_802,11.22,0


In [39]:
# observation de la moyenne / mediane / répartition des valeurs
df_categ_zero.describe()

,price,categ
count,2308.000000,2308.0
mean,11.732795,0.0
std,7.565755,0.0
min,0.620000,0.0
25%,5.587500,0.0
50%,10.320000,0.0
75%,16.655000,0.0
max,40.990000,0.0


In [40]:
# remplacement des valeur manquante prix par la médiane
df_trans_prod["price"] = df_trans_prod["price"].fillna(df_categ_zero["price"].median())
df_trans_prod

,id_prod,date,session_id,client_id,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0.0
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0.0
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0.0
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0.0
...,...,...,...,...,...,...
679348,0_1624,NaT,NaN,NaN,24.50,0.0
679349,2_86,NaT,NaN,NaN,132.36,2.0
679350,0_299,NaT,NaN,NaN,22.99,0.0
679351,0_510,NaT,NaN,NaN,23.66,0.0


In [41]:
# vérif que le prix nul été remplacé par la médiane des prix de la categ 0
df_trans_prod.loc[df_trans_prod["id_prod"]== "0_2245"]

,id_prod,date,session_id,client_id,price,categ
542560,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,10.32,0.0
542561,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,10.32,0.0
542562,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,10.32,0.0
542563,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,10.32,0.0
542564,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,10.32,0.0
...,...,...,...,...,...,...
542776,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,10.32,0.0
542777,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,10.32,0.0
542778,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,10.32,0.0
542779,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,10.32,0.0


les produits sans référence client_id / session_id sont des produits qui n'ont pas fait l'objet de commande  
les 21 produits n'ont jamais fait l'objet d'une vente

In [42]:
# suppression des lignes des produits n'ayant pas fait l'objet de commande
df_trans_prod.dropna(inplace= True)
df_trans_prod

,id_prod,date,session_id,client_id,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0.0
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0.0
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0.0
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0.0
...,...,...,...,...,...,...
679327,0_1279,2021-06-23 06:50:48.888907,s_52563,c_1025,20.99,0.0
679328,0_1683,2021-12-24 04:37:46.407713,s_139091,c_5508,2.99,0.0
679329,0_1379,2021-10-16 09:19:09.546713,s_105547,c_5425,2.99,0.0
679330,0_886,2021-03-17 03:34:14.461358,s_7410,c_7219,21.82,0.0


In [43]:
# vérif que la modif a été prise en compte
df_trans_prod.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
price         0
categ         0
dtype: int64

## 3.2 jointure entre df_trans_prod et df_custom

### 3.2.1 jointure et vérification des données

In [44]:
# jointure externe sur la colonne commune client_id
df_complet = pd.merge(df_trans_prod, df_custom, on="client_id", how="outer")
df_complet

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,f,1986
1,0_1518,2021-07-20 13:21:29.043970,s_64849,c_103,4.18,0.0,f,1986
2,0_1518,2022-08-20 13:21:29.043970,s_255965,c_103,4.18,0.0,f,1986
3,0_1418,2022-06-18 01:49:37.823274,s_225411,c_103,8.57,0.0,f,1986
4,0_1418,2021-08-18 01:49:37.823274,s_77214,c_103,8.57,0.0,f,1986
...,...,...,...,...,...,...,...,...
679348,NaN,NaT,NaN,c_862,NaN,NaN,f,1956
679349,NaN,NaT,NaN,c_7584,NaN,NaN,f,1960
679350,NaN,NaT,NaN,c_90,NaN,NaN,m,2001
679351,NaN,NaT,NaN,c_587,NaN,NaN,m,1993


In [45]:
# observation des valeurs nulles après jointure
df_complet.isnull().sum()

id_prod       21
date          21
session_id    21
client_id      0
price         21
categ         21
sex            0
birth          0
dtype: int64

In [46]:
df_nul_date = df_complet[df_complet["id_prod"].isnull()]
df_nul_date

,id_prod,date,session_id,client_id,price,categ,sex,birth
679332,NaN,NaT,NaN,c_8253,NaN,NaN,f,2001
679333,NaN,NaT,NaN,c_3789,NaN,NaN,f,1997
679334,NaN,NaT,NaN,c_4406,NaN,NaN,f,1998
679335,NaN,NaT,NaN,c_2706,NaN,NaN,f,1967
679336,NaN,NaT,NaN,c_3443,NaN,NaN,m,1959
679337,NaN,NaT,NaN,c_4447,NaN,NaN,m,1956
679338,NaN,NaT,NaN,c_3017,NaN,NaN,f,1992
679339,NaN,NaT,NaN,c_4086,NaN,NaN,f,1992
679340,NaN,NaT,NaN,c_6930,NaN,NaN,m,2004
679341,NaN,NaT,NaN,c_4358,NaN,NaN,m,1999


les valeurs manquantes en id_prod correspondent à 21 clients qui n'ont pas fait d'achat sur le site ou dont les achats n'ont pas été pris en compte

### 3.2.2 traitement des erreurs suite jointure

In [47]:
# supression des clients n'ayant pas fait de commande
df_complet.dropna(inplace=True)

In [48]:
#verif que toute les valeurs nulles ont été supprimée
df_complet.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
price         0
categ         0
sex           0
birth         0
dtype: int64

# 4 ajouts de colonne au dataframe

## 4.1 ajout d'un colonne age des clients

In [49]:
# calcul de l'age des clients avec ajout d'une colonne age
an = datetime.now().year
df_complet["age"] = an - df_complet["birth"]
df_complet

,id_prod,date,session_id,client_id,price,categ,sex,birth,age
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,f,1986,37
1,0_1518,2021-07-20 13:21:29.043970,s_64849,c_103,4.18,0.0,f,1986,37
2,0_1518,2022-08-20 13:21:29.043970,s_255965,c_103,4.18,0.0,f,1986,37
3,0_1418,2022-06-18 01:49:37.823274,s_225411,c_103,8.57,0.0,f,1986,37
4,0_1418,2021-08-18 01:49:37.823274,s_77214,c_103,8.57,0.0,f,1986,37
...,...,...,...,...,...,...,...,...,...
679327,2_147,2021-07-23 07:01:38.963669,s_65994,c_4391,181.99,2.0,f,2001,22
679328,0_142,2022-03-25 18:07:25.880052,s_184472,c_1232,19.85,0.0,f,1960,63
679329,0_142,2021-09-25 18:07:25.880052,s_95415,c_1232,19.85,0.0,f,1960,63
679330,2_205,2021-03-11 00:11:32.016264,s_4605,c_7534,100.99,2.0,m,1992,31


## 4.2 ajout d'une colonne classe d'age

In [50]:
df_complet["classe_age"] = "18-30"
df_complet["classe_age"].loc[df_complet[df_complet["age"]>=30].index] = "31-40"
df_complet["classe_age"].loc[df_complet[df_complet["age"]>=40].index] = "41-50"
df_complet["classe_age"].loc[df_complet[df_complet["age"]>=50].index] = "51-60"
df_complet["classe_age"].loc[df_complet[df_complet["age"]>=60].index] = "61-70"
df_complet["classe_age"].loc[df_complet[df_complet["age"]>=70].index] = "71-80"
df_complet["classe_age"].loc[df_complet[df_complet["age"]>=80].index] = "81 et +"
df_complet

,id_prod,date,session_id,client_id,price,categ,sex,birth,age,classe_age
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0.0,f,1986,37,31-40
1,0_1518,2021-07-20 13:21:29.043970,s_64849,c_103,4.18,0.0,f,1986,37,31-40
2,0_1518,2022-08-20 13:21:29.043970,s_255965,c_103,4.18,0.0,f,1986,37,31-40
3,0_1418,2022-06-18 01:49:37.823274,s_225411,c_103,8.57,0.0,f,1986,37,31-40
4,0_1418,2021-08-18 01:49:37.823274,s_77214,c_103,8.57,0.0,f,1986,37,31-40
...,...,...,...,...,...,...,...,...,...,...
679327,2_147,2021-07-23 07:01:38.963669,s_65994,c_4391,181.99,2.0,f,2001,22,18-30
679328,0_142,2022-03-25 18:07:25.880052,s_184472,c_1232,19.85,0.0,f,1960,63,61-70
679329,0_142,2021-09-25 18:07:25.880052,s_95415,c_1232,19.85,0.0,f,1960,63,61-70
679330,2_205,2021-03-11 00:11:32.016264,s_4605,c_7534,100.99,2.0,m,1992,31,31-40


In [51]:
df_complet.to_csv("df_complet.csv", index=False)

In [52]:
df_complet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679332 entries, 0 to 679331
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679332 non-null  object        
 1   date        679332 non-null  datetime64[ns]
 2   session_id  679332 non-null  object        
 3   client_id   679332 non-null  object        
 4   price       679332 non-null  float64       
 5   categ       679332 non-null  float64       
 6   sex         679332 non-null  object        
 7   birth       679332 non-null  int64         
 8   age         679332 non-null  int64         
 9   classe_age  679332 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 73.1+ MB


# 5 création d'un dataframe pour pouvoir répondre aux différentes demandes

In [53]:
# création d'un df groupé par session pour avoir l'heure de début et l'heure de fin de la session 
df_session1 = df_complet.groupby("session_id")["date"].agg(["min", "max"]).reset_index()
df_session1

,session_id,min,max
0,s_1,2021-03-01 00:01:07.843138,2021-03-01 00:01:07.843138
1,s_10,2021-03-01 00:10:33.163037,2021-03-01 00:10:33.163037
2,s_100,2021-03-01 04:12:43.572994,2021-03-01 04:43:58.025677
3,s_1000,2021-03-03 02:38:09.568389,2021-03-03 03:25:12.320225
4,s_10000,2021-03-22 17:46:05.819130,2021-03-22 18:15:03.831240
...,...,...,...
342310,s_99994,2021-10-04 18:34:24.208113,2021-10-04 18:56:23.112236
342311,s_99995,2021-10-04 18:35:32.201073,2021-10-04 18:35:32.201073
342312,s_99996,2021-10-04 18:38:36.333661,2021-10-04 18:58:49.228317
342313,s_99997,2021-10-04 18:45:38.003516,2021-10-04 18:45:38.003516


In [54]:
# détermination de la durée des session
df_session1["duree"] = (df_session1["max"]- df_session1["min"]).dt.seconds/60
df_session1

,session_id,min,max,duree
0,s_1,2021-03-01 00:01:07.843138,2021-03-01 00:01:07.843138,0.000000
1,s_10,2021-03-01 00:10:33.163037,2021-03-01 00:10:33.163037,0.000000
2,s_100,2021-03-01 04:12:43.572994,2021-03-01 04:43:58.025677,31.233333
3,s_1000,2021-03-03 02:38:09.568389,2021-03-03 03:25:12.320225,47.033333
4,s_10000,2021-03-22 17:46:05.819130,2021-03-22 18:15:03.831240,28.966667
...,...,...,...,...
342310,s_99994,2021-10-04 18:34:24.208113,2021-10-04 18:56:23.112236,21.966667
342311,s_99995,2021-10-04 18:35:32.201073,2021-10-04 18:35:32.201073,0.000000
342312,s_99996,2021-10-04 18:38:36.333661,2021-10-04 18:58:49.228317,20.200000
342313,s_99997,2021-10-04 18:45:38.003516,2021-10-04 18:45:38.003516,0.000000


In [55]:
# création d'un df pour connaitre le nombre d'article et la valeur du panier par session
df_session2 = df_complet.groupby("session_id")["price"].agg(["count", "sum"]).reset_index()
df_session2

,session_id,count,sum
0,s_1,1,11.99
1,s_10,1,26.99
2,s_100,2,33.72
3,s_1000,4,39.22
4,s_10000,3,41.49
...,...,...,...
342310,s_99994,3,28.92
342311,s_99995,1,19.84
342312,s_99996,4,56.27
342313,s_99997,1,6.99


In [56]:
# création d'un df pour récupérer les info clients
df_session3 = df_complet[["session_id", "client_id", "id_prod", "age", "classe_age", "sex", "categ"]].reset_index(drop=True)
df_session3.drop_duplicates(subset=["session_id"], inplace=True)
df_session3

,session_id,client_id,id_prod,age,classe_age,sex,categ
0,s_211425,c_103,0_1518,37,31-40,f,0.0
1,s_64849,c_103,0_1518,37,31-40,f,0.0
2,s_255965,c_103,0_1518,37,31-40,f,0.0
3,s_225411,c_103,0_1418,37,31-40,f,0.0
4,s_77214,c_103,0_1418,37,31-40,f,0.0
...,...,...,...,...,...,...,...
679327,s_65994,c_4391,2_147,22,18-30,f,2.0
679328,s_184472,c_1232,0_142,63,61-70,f,0.0
679329,s_95415,c_1232,0_142,63,61-70,f,0.0
679330,s_4605,c_7534,2_205,31,31-40,m,2.0


In [57]:
# jointure des 2 premier df créé
df_session4 = pd.merge(df_session1, df_session2, on= "session_id", how="inner")
df_session4

,session_id,min,max,duree,count,sum
0,s_1,2021-03-01 00:01:07.843138,2021-03-01 00:01:07.843138,0.000000,1,11.99
1,s_10,2021-03-01 00:10:33.163037,2021-03-01 00:10:33.163037,0.000000,1,26.99
2,s_100,2021-03-01 04:12:43.572994,2021-03-01 04:43:58.025677,31.233333,2,33.72
3,s_1000,2021-03-03 02:38:09.568389,2021-03-03 03:25:12.320225,47.033333,4,39.22
4,s_10000,2021-03-22 17:46:05.819130,2021-03-22 18:15:03.831240,28.966667,3,41.49
...,...,...,...,...,...,...
342310,s_99994,2021-10-04 18:34:24.208113,2021-10-04 18:56:23.112236,21.966667,3,28.92
342311,s_99995,2021-10-04 18:35:32.201073,2021-10-04 18:35:32.201073,0.000000,1,19.84
342312,s_99996,2021-10-04 18:38:36.333661,2021-10-04 18:58:49.228317,20.200000,4,56.27
342313,s_99997,2021-10-04 18:45:38.003516,2021-10-04 18:45:38.003516,0.000000,1,6.99


In [58]:
# regroupement de tout les df créé
df_sess_compl = pd.merge(df_session3, df_session4, on="session_id", how="inner")
df_sess_compl

,session_id,client_id,id_prod,age,classe_age,sex,categ,min,max,duree,count,sum
0,s_211425,c_103,0_1518,37,31-40,f,0.0,2022-05-20 13:21:10.122134,2022-05-20 13:21:29.043970,0.300000,2,21.17
1,s_64849,c_103,0_1518,37,31-40,f,0.0,2021-07-20 13:21:10.122134,2021-07-20 13:21:29.043970,0.300000,2,21.17
2,s_255965,c_103,0_1518,37,31-40,f,0.0,2022-08-20 13:21:10.122134,2022-08-20 13:21:29.043970,0.300000,2,21.17
3,s_225411,c_103,0_1418,37,31-40,f,0.0,2022-06-18 01:26:20.939798,2022-06-18 01:49:37.823274,23.266667,3,38.55
4,s_77214,c_103,0_1418,37,31-40,f,0.0,2021-08-18 01:26:20.939798,2021-08-18 01:49:37.823274,23.266667,3,38.55
...,...,...,...,...,...,...,...,...,...,...,...,...
342310,s_65994,c_4391,2_147,22,18-30,f,2.0,2021-07-23 07:01:38.963669,2021-07-23 07:01:38.963669,0.000000,1,181.99
342311,s_184472,c_1232,0_142,63,61-70,f,0.0,2022-03-25 18:07:25.880052,2022-03-25 18:07:25.880052,0.000000,1,19.85
342312,s_95415,c_1232,0_142,63,61-70,f,0.0,2021-09-25 18:07:25.880052,2021-09-25 18:07:25.880052,0.000000,1,19.85
342313,s_4605,c_7534,2_205,31,31-40,m,2.0,2021-03-11 00:11:32.016264,2021-03-11 00:11:32.016264,0.000000,1,100.99


In [59]:
# changement des noms de colonne
df_sess_compl.rename(columns={"min":"debut_sess", "max":"fin_sess", "count":"fréquence", "sum":"CA_sess"}, inplace=True)

In [60]:
df_sess_compl.head()

,session_id,client_id,id_prod,age,classe_age,sex,categ,debut_sess,fin_sess,duree,fréquence,CA_sess
0,s_211425,c_103,0_1518,37,31-40,f,0.0,2022-05-20 13:21:10.122134,2022-05-20 13:21:29.043970,0.300000,2,21.17
1,s_64849,c_103,0_1518,37,31-40,f,0.0,2021-07-20 13:21:10.122134,2021-07-20 13:21:29.043970,0.300000,2,21.17
2,s_255965,c_103,0_1518,37,31-40,f,0.0,2022-08-20 13:21:10.122134,2022-08-20 13:21:29.043970,0.300000,2,21.17
3,s_225411,c_103,0_1418,37,31-40,f,0.0,2022-06-18 01:26:20.939798,2022-06-18 01:49:37.823274,23.266667,3,38.55
4,s_77214,c_103,0_1418,37,31-40,f,0.0,2021-08-18 01:26:20.939798,2021-08-18 01:49:37.823274,23.266667,3,38.55


In [61]:
df_sess_compl.to_csv("df_sess_compl.csv", index=False)

In [62]:
df_sess_compl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342315 entries, 0 to 342314
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   session_id  342315 non-null  object        
 1   client_id   342315 non-null  object        
 2   id_prod     342315 non-null  object        
 3   age         342315 non-null  int64         
 4   classe_age  342315 non-null  object        
 5   sex         342315 non-null  object        
 6   categ       342315 non-null  float64       
 7   debut_sess  342315 non-null  datetime64[ns]
 8   fin_sess    342315 non-null  datetime64[ns]
 9   duree       342315 non-null  float64       
 10  fréquence   342315 non-null  int64         
 11  CA_sess     342315 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(5)
memory usage: 34.0+ MB
